# Async SQLAlchemy

As part of the 1.4/2.0 SQAlchemy release, SQLAlchemy now supports async operations.

Async SQLAlchemy isn't a performance panacea - both SQLAlchemy and your database are built to handle out-of-the-box queries very well, but it does help with the [function colour](https://journal.stuffwithstuff.com/2015/02/01/what-color-is-your-function/) problem of having to divide your world into async and non-async functions.

## Async SQLAlchemy Core

Let's start with revisiting our Core layer

In [5]:
from sqlalchemy.ext.asyncio import create_async_engine
import sqlalchemy as sa
import aiosqlite

Our engine is now an AsyncEngine, but the table definition stays the same. Note that we do need to have an async-compatible db driver installed to be able to use async. In this case, we're using aiosqlite instead of the built-in sqlite library

In [4]:
engine = create_async_engine("sqlite+aiosqlite:///async_local.db", future=True)

In [6]:
meta = sa.MetaData()
table = sa.Table("cars", meta, 
                sa.Column("car_id", sa.Integer, primary_key=True),
                 sa.Column("make", sa.String),
                 sa.Column("year", sa.Integer),
                 sa.Column("mileage", sa.Integer)
                )

DDL has to be run synchronously when inside the async conneciton, so every `conn` has a `run_sync` method

In [7]:
async with engine.begin() as conn:
    await conn.run_sync(meta.create_all)

Jupyter natively support async (so we don't have to use `asyncio.run` to run our async functions on the event loop

In [8]:
cars_data = [
    {"make": "Honda Civic", "year": 2012, "mileage": 20_000},
    {"make": "Toyota Prius", "year": 2020, "mileage": 10_000},
    {"make": "Tesla Model 3", "year": 2019, "mileage": 5_000}
]

In [9]:
async with engine.begin() as conn:
    await conn.execute(table.insert(), cars_data)

In [10]:
import pandas as pd

In [13]:
async with engine.connect() as conn:
    sql = sa.select(table).where(table.c.mileage > 6000)
    r = await conn.execute(sql)

# This is a blocking operation
df = pd.DataFrame(r)

In [14]:
df

,car_id,make,year,mileage
0,1,Honda Civic,2012,20000
1,2,Toyota Prius,2020,10000


## Async SQLAlchemy 